In [2]:
# grid search sarima hyperparameters for monthly mean sales 
import time
import pandas as pd
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

In [3]:
# one-step sarima forecast
def sarima_forecast(history, config):
    order, sorder, trend = config
    # define model
    model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
    # fit model
    model_fit = model.fit(disp=False)
    # make one step forecast
    yhat = model_fit.predict(len(history), len(history))
    return yhat[0]

# root mean squared error or rmse
def measure_rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
    return data[:-n_test], data[-n_test:]

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
    predictions = list()
    # split dataset
    train, test = train_test_split(data, n_test)
    # seed history with training dataset
    history = [x for x in train]
    # step over each time-step in the test set
    for i in range(len(test)):
        # fit model and make forecast for history
        yhat = sarima_forecast(history, cfg)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        history.append(test[i])
    # estimate prediction error
    error = measure_rmse(test, predictions)
    return error

# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
    result = None
    # convert config to a key
    key = str(cfg)
    # show all warnings and fail on exception if debugging
    if debug:
        result = walk_forward_validation(data, n_test, cfg)
    else:
        # one failure during model validation suggests an unstable config
        try:
            # never show warnings when grid searching, too noisy
            with catch_warnings():
                filterwarnings("ignore")
                result = walk_forward_validation(data, n_test, cfg)
        except:
            error = None
    # check for an interesting result
    if result is not None:
        print(' > Model[%s] %.3f' % (cfg, result))    # print(' > Model[%s] %.3f' % (key, result))  -- revised from
    return (cfg, result)

# grid search configs
def grid_search(data, cfg_list, n_test):
    scores = None
    scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
    # remove empty results
    scores = [r for r in scores if r[1] != None]
    # sort configs by error, asc
    scores.sort(key=lambda tup: tup[1])
    return scores

# create a set of sarima configs to try
def sarima_configs(seasonal=[0]):
    models = list()
    # define config lists
    p_params = [0, 1, 2] # [0, 1, 2]
    d_params = [0, 1] # [0, 1]
    q_params = [0, 1, 2] # [0, 1, 2]
    t_params = ['c','t']   # ['n','c','t','ct']
    P_params = [0, 1, 2] # [0, 1, 2]
    D_params = [0, 1]
    Q_params = [0, 1, 2] # [0, 1, 2]
    m_params = seasonal
    # create config instances
    for p in p_params:
        for d in d_params:
            for q in q_params:
                for t in t_params:
                    for P in P_params:
                        for D in D_params:
                            for Q in Q_params:
                                for m in m_params:
                                    cfg = [(p,d,q), (P,D,Q,m), t]
                                    models.append(cfg)
    return models

In [5]:
# load dataset
df = pd.read_csv('../data-processed/orderproducts_top20.csv', parse_dates=[1], infer_datetime_format=True)
prod_monthly = pd.crosstab(df['order_date'], df['product_sku']).resample('M').sum()
prod_monthly = prod_monthly['2018-01':'2021-03']
items = prod_monthly.columns.values

#### Find the optimal p,d,q,P,D,Q,t for SARIMA Modelling

In [6]:
start_time = time.time()
data = prod_monthly['EFX-FLY-BLK'].values
# data split
n_test = 12
# model configs
cfg_list = sarima_configs(seasonal=[12])  # 12 months
# grid search
scores = grid_search(data, cfg_list, n_test)
# list top 10 configs
for cfg, error in scores[:10]:
    print(cfg, error)

pdq = pd.DataFrame(scores, columns=['pdq_config', 'rmse'])
#pdq.to_csv('../data/pqd_scores.csv')
end_time = time.time()
print('time taken :', round(end_time-start_time,1),'secs')

 > Model[[(0, 0, 0), (0, 0, 0, 12), 'c']] 10.269
 > Model[[(0, 0, 0), (0, 0, 1, 12), 'c']] 5.786
 > Model[[(0, 0, 0), (0, 0, 2, 12), 'c']] 2.620
 > Model[[(0, 0, 0), (0, 1, 0, 12), 'c']] 6.695
 > Model[[(0, 0, 0), (0, 1, 1, 12), 'c']] 4.920
 > Model[[(0, 0, 0), (0, 1, 2, 12), 'c']] 6.062
 > Model[[(0, 0, 0), (1, 0, 0, 12), 'c']] 3.583
 > Model[[(0, 0, 0), (1, 0, 1, 12), 'c']] 3.328
 > Model[[(0, 0, 0), (1, 0, 2, 12), 'c']] 2.812
 > Model[[(0, 0, 0), (1, 1, 0, 12), 'c']] 5.125
 > Model[[(0, 0, 0), (1, 1, 1, 12), 'c']] 6.236
 > Model[[(0, 0, 0), (1, 1, 2, 12), 'c']] 8.766
 > Model[[(0, 0, 0), (2, 0, 0, 12), 'c']] 3.847
 > Model[[(0, 0, 0), (2, 0, 1, 12), 'c']] 3.860
 > Model[[(0, 0, 0), (2, 0, 2, 12), 'c']] 4.901
 > Model[[(0, 0, 0), (2, 1, 0, 12), 'c']] 6.642
 > Model[[(0, 0, 0), (2, 1, 1, 12), 'c']] 6.642
 > Model[[(0, 0, 0), (2, 1, 2, 12), 'c']] 5.818
 > Model[[(0, 0, 0), (0, 0, 0, 12), 't']] 10.555
 > Model[[(0, 0, 0), (0, 0, 1, 12), 't']] 6.627
 > Model[[(0, 0, 0), (0, 0, 2, 12), 't

 > Model[[(0, 1, 1), (1, 1, 0, 12), 't']] 8.726
 > Model[[(0, 1, 1), (1, 1, 1, 12), 't']] 9.405
 > Model[[(0, 1, 1), (1, 1, 2, 12), 't']] 10.930
 > Model[[(0, 1, 1), (2, 0, 0, 12), 't']] 6.527
 > Model[[(0, 1, 1), (2, 0, 1, 12), 't']] 3.568
 > Model[[(0, 1, 1), (2, 0, 2, 12), 't']] 9.238
 > Model[[(0, 1, 1), (2, 1, 0, 12), 't']] 12.555
 > Model[[(0, 1, 1), (2, 1, 1, 12), 't']] 12.638
 > Model[[(0, 1, 1), (2, 1, 2, 12), 't']] 10.930
 > Model[[(0, 1, 2), (0, 0, 0, 12), 'c']] 3.546
 > Model[[(0, 1, 2), (0, 0, 1, 12), 'c']] 4.069
 > Model[[(0, 1, 2), (0, 0, 2, 12), 'c']] 5.528
 > Model[[(0, 1, 2), (0, 1, 0, 12), 'c']] 6.428
 > Model[[(0, 1, 2), (0, 1, 1, 12), 'c']] 7.486
 > Model[[(0, 1, 2), (0, 1, 2, 12), 'c']] 6.747
 > Model[[(0, 1, 2), (1, 0, 0, 12), 'c']] 3.398
 > Model[[(0, 1, 2), (1, 0, 1, 12), 'c']] 3.365
 > Model[[(0, 1, 2), (1, 0, 2, 12), 'c']] 6.324
 > Model[[(0, 1, 2), (1, 1, 0, 12), 'c']] 7.985
 > Model[[(0, 1, 2), (1, 1, 1, 12), 'c']] 7.126
 > Model[[(0, 1, 2), (1, 1, 2, 12), 

 > Model[[(1, 1, 0), (0, 0, 0, 12), 't']] 3.651
 > Model[[(1, 1, 0), (0, 0, 1, 12), 't']] 3.635
 > Model[[(1, 1, 0), (0, 0, 2, 12), 't']] 3.675
 > Model[[(1, 1, 0), (0, 1, 0, 12), 't']] 4.754
 > Model[[(1, 1, 0), (0, 1, 1, 12), 't']] 7.489
 > Model[[(1, 1, 0), (0, 1, 2, 12), 't']] 5.717
 > Model[[(1, 1, 0), (1, 0, 0, 12), 't']] 3.442
 > Model[[(1, 1, 0), (1, 0, 1, 12), 't']] 3.625
 > Model[[(1, 1, 0), (1, 0, 2, 12), 't']] 5.704
 > Model[[(1, 1, 0), (1, 1, 0, 12), 't']] 8.872
 > Model[[(1, 1, 0), (1, 1, 1, 12), 't']] 7.728
 > Model[[(1, 1, 0), (1, 1, 2, 12), 't']] 5.717
 > Model[[(1, 1, 0), (2, 0, 0, 12), 't']] 5.614
 > Model[[(1, 1, 0), (2, 0, 1, 12), 't']] 10.610
 > Model[[(1, 1, 0), (2, 0, 2, 12), 't']] 10.610
 > Model[[(1, 1, 0), (2, 1, 0, 12), 't']] 6.216
 > Model[[(1, 1, 0), (2, 1, 1, 12), 't']] 5.717
 > Model[[(1, 1, 0), (2, 1, 2, 12), 't']] 5.717
 > Model[[(1, 1, 1), (0, 0, 0, 12), 'c']] 4.193
 > Model[[(1, 1, 1), (0, 0, 1, 12), 'c']] 3.224
 > Model[[(1, 1, 1), (0, 0, 2, 12), 'c

 > Model[[(2, 0, 2), (1, 0, 0, 12), 'c']] 3.312
 > Model[[(2, 0, 2), (1, 0, 1, 12), 'c']] 3.466
 > Model[[(2, 0, 2), (1, 0, 2, 12), 'c']] 55.760
 > Model[[(2, 0, 2), (1, 1, 0, 12), 'c']] 12204365747220586496.000
 > Model[[(2, 0, 2), (1, 1, 1, 12), 'c']] 552833015722.207
 > Model[[(2, 0, 2), (1, 1, 2, 12), 'c']] 4.815
 > Model[[(2, 0, 2), (2, 0, 0, 12), 'c']] 5.876
 > Model[[(2, 0, 2), (2, 0, 1, 12), 'c']] 5.562
 > Model[[(2, 0, 2), (2, 0, 2, 12), 'c']] 7.894
 > Model[[(2, 0, 2), (2, 1, 0, 12), 'c']] 6.272
 > Model[[(2, 0, 2), (2, 1, 1, 12), 'c']] 4.686
 > Model[[(2, 0, 2), (2, 1, 2, 12), 'c']] 4.815
 > Model[[(2, 0, 2), (0, 0, 0, 12), 't']] 2.990
 > Model[[(2, 0, 2), (0, 0, 1, 12), 't']] 5.438
 > Model[[(2, 0, 2), (0, 0, 2, 12), 't']] 4.635
 > Model[[(2, 0, 2), (0, 1, 0, 12), 't']] 5.110
 > Model[[(2, 0, 2), (0, 1, 1, 12), 't']] 6.972
 > Model[[(2, 0, 2), (0, 1, 2, 12), 't']] 10.629
 > Model[[(2, 0, 2), (1, 0, 0, 12), 't']] 3.218
 > Model[[(2, 0, 2), (1, 0, 1, 12), 't']] 5.081
 > Model

#### Forecast the last 3 month sales for the top 20 items

In [9]:
prod_monthly = pd.crosstab(df['order_date'], df['product_sku']).resample('M').sum()
prod_monthly = prod_monthly['2018-01':'2021-03']
items = prod_monthly.columns.values

# items are the names of the top20 items
test_predict = []
mse_list = []
results =  pd.DataFrame()
#order, sorder, trend = [(1, 0, 1), (0, 0, 0, 12), 'c']   # scores[0][0]
order, sorder, trend = scores[0][0]

for item in items:
    data = prod_monthly[item]
    train = data.iloc[:-3]
    test  = data.iloc[-3:]
    
    model = model = SARIMAX(train, order=order, seasonal_order=sorder, 
                            trend=trend, enforce_stationarity=False, enforce_invertibility=False)
    model_fit = model.fit(disp=False)
    forecast = model_fit.predict(len(train), len(train)+3)
    adj_forecast = [ 0 if x < 0 else int(round(x)) for x in forecast ]
    item_name = [item for x in range(3)]
    res = pd.DataFrame(zip(np.array(test), np.array(adj_forecast), item_name), 
                       index=['m+1','m+2','m+3'], columns=['test', 'predict', 'item'])
    results = pd.concat([results,res], axis=0)
print(results)

C:\Anaconda3\envs\powlook\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:868: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  ' zeros.' % warning_description)


     test  predict             item
m+1     2        4      EFX-FLY-BLK
m+2     2        4      EFX-FLY-BLK
m+3     5        4      EFX-FLY-BLK
m+1     1        2       M80-2B-BLK
m+2     1        2       M80-2B-BLK
m+3     1        2       M80-2B-BLK
m+1     5        3       M80-2G-BLK
m+2     4        3       M80-2G-BLK
m+3     5        3       M80-2G-BLK
m+1     0        2       M80-AC-BLK
m+2     0        2       M80-AC-BLK
m+3     1        2       M80-AC-BLK
m+1     3        2       M80-AD-BLK
m+2     1        2       M80-AD-BLK
m+3     0        2       M80-AD-BLK
m+1     3        1    M80-BTY-BLK-L
m+2     1        1    M80-BTY-BLK-L
m+3     1        1    M80-BTY-BLK-L
m+1     0        2    M80-BTY-BLK-S
m+2     2        2    M80-BTY-BLK-S
m+3     1        2    M80-BTY-BLK-S
m+1     3        1       M80-EB-BLK
m+2     1        1       M80-EB-BLK
m+3     1        1       M80-EB-BLK
m+1     2        2       M80-EG-BLK
m+2     0        2       M80-EG-BLK
m+3     3        2       M80

In [10]:
results.to_csv('../data-processed/top20forecasts_SARIMA_01to03.csv')